### Exercises

Using the repo setup directions, setup a new local and remote repository named spark-exercises. The local version of your repo should live inside of ~/codeup-data-science. This repo should be named spark-exercises

Save this work in your spark-exercises repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named spark101 for this exercise.

In [3]:
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_extract, regexp_replace

In [4]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/20 22:56:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [6]:
#create panda dataframe
df = pd.DataFrame({"language":["python","sql","ruby","c","java","c++"]})
df

,language
0,python
1,sql
2,ruby
3,c
4,java
5,c++


In [7]:
#now create spark dataframe 
spark_df= spark.createDataFrame(df)
spark_df.show()

+--------+
|language|
+--------+
|  python|
|     sql|
|    ruby|
|       c|
|    java|
|     c++|
+--------+



In [8]:
#view schema
spark_df.printSchema()

root
 |-- language: string (nullable = true)



In [9]:
#shape of dataframe, not as simple as pandas
#import count
from pyspark.sql.functions import count
spark_df.count(), len(spark_df.columns)

(6, 1)

In [10]:
#show first 5 records
spark_df.show(5)

+--------+
|language|
+--------+
|  python|
|     sql|
|    ruby|
|       c|
|    java|
+--------+
only showing top 5 rows



### 2.Load the mpg dataset as a spark dataframe.

In [11]:
#import data where mpg resides
from pydataset import data

In [12]:
#get mpg pyspark data
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)                         

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



#### a. Create 1 column of output that contains a message like the one below:
    The 1999 audi a4 has a 4 cylinder engine

In [31]:
from pyspark.sql.functions import concat
mpg.select(concat(lit("The "), "year", lit(" "), "manufacturer",lit(" "), "model", lit(" has a "), "cyl", lit(" cylinder engine")).alias("concat")).show(5,truncate = False)

+----------------------------------------+
|concat                                  |
+----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 2008 audi a4 has a 4 cylinder engine|
|The 1999 audi a4 has a 6 cylinder engine|
+----------------------------------------+
only showing top 5 rows



#### b. Transform the trans column so that it only contains either manual or auto.


In [14]:
mpg.select("trans").show(1)

+--------+
|   trans|
+--------+
|auto(l5)|
+--------+
only showing top 1 row



In [27]:
mpg.select("trans",regexp_extract("trans" , r"(\w+)", 1).alias("transmission")).show(4)

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
+----------+------------+
only showing top 4 rows



In [29]:
mpg.select(regexp_extract("trans" , r"(\w+)", 1).alias("transmission")).show(4)

+------------+
|transmission|
+------------+
|        auto|
|      manual|
|      manual|
|        auto|
+------------+
only showing top 4 rows



### 3. Load the tips dataset as a spark dataframe.

In [30]:
data("tips")

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3
241,27.18,2.00,Female,Yes,Sat,Dinner,2
242,22.67,2.00,Male,Yes,Sat,Dinner,2
243,17.82,1.75,Male,No,Sat,Dinner,2
